In [72]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        pass
#        print(os.path.join(dirname, filename))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [73]:
# 코드 출처 주석에 바로 적기

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly # 구매 아이템 정보
import matplotlib.image as mpimg
# 처음엔 overall view하는 것이 세세하게 연령별로 group view 하는 것보다 파악이 빠르다
import plotly.graph_objects as go
import random
import plotly.offline as po
from sklearn.feature_extraction.text import TfidfVectorizer
import gc 
from sklearn.model_selection import train_test_split
import pandas as pd

In [5]:
%%time

base_path = '../input/h-and-m-personalized-fashion-recommendations/'
csv_train = f'{base_path}transactions_train.csv'
csv_sub = f'{base_path}sample_submission.csv'
csv_users = f'{base_path}customers.csv'
csv_items = f'{base_path}articles.csv'

debug=True
if debug:
    df = pd.read_csv(csv_train, dtype={'article_id': str}, parse_dates=['t_dat'], nrows=300000)
    df_sub = pd.read_csv(csv_sub)
    dfu = pd.read_csv(csv_users)
    dfi = pd.read_csv(csv_items, dtype={'article_id': str})
else:
    df = pd.read_csv(csv_train, dtype={'article_id': str}, parse_dates=['t_dat'])
    df_sub = pd.read_csv(csv_sub)
    dfu = pd.read_csv(csv_users)
    dfi = pd.read_csv(csv_items, dtype={'article_id': str})

Wall time: 3.9 s


In [9]:
df['customer_id']=  df['customer_id'].apply(lambda x :  int( x[-16:],16))


TypeError: 'int' object is not subscriptable

In [10]:
df['customer_id'] = df['customer_id'].astype('int64')

In [11]:
df.t_dat = pd.to_datetime(df.t_dat)

In [12]:
df = df[['t_dat','customer_id','article_id']]

In [13]:
df.to_parquet('train.pqt',index=False)

In [14]:
print(df.shape)

(300000, 3)


In [15]:
df.head()

,t_dat,customer_id,article_id
0,2018-09-20,-6846340800584936,0663713001
1,2018-09-20,-6846340800584936,0541518023
2,2018-09-20,-8334631767138808638,0505221004
3,2018-09-20,-8334631767138808638,0685687003
4,2018-09-20,-8334631767138808638,0685687004


In [16]:
# Find Each Customer's Last Week of Purchases

In [17]:
tmp = df.groupby('customer_id').t_dat.max().reset_index()

In [18]:
tmp.columns =['customer_id', 'max_dat']

In [19]:
df = df.merge(tmp, on ='customer_id' , how='left')

In [20]:
df['diff_dat'] = (df.max_dat - df.t_dat).dt.days

In [21]:
df = df.loc[df['diff_dat']<=6]

In [22]:
print('Train shape: ', df.shape)

Train shape:  (300000, 5)


In [23]:
tmp = df.groupby(['customer_id','article_id'])['t_dat'].agg('count').reset_index()

In [24]:
tmp.columns = ['customer_id','article_id','ct']

In [25]:
df = df.merge(tmp, on=['customer_id','article_id'],how='left')

In [26]:
df = df.sort_values(['ct','t_dat'] , ascending=False)

In [27]:
df =  df.drop_duplicates(['customer_id','article_id'])

In [28]:
df = df.sort_values(['ct','t_dat'], ascending=False)

In [29]:
df_ = pd.read_csv(csv_train, dtype={'article_id': str}, parse_dates=['t_dat'], nrows=300000)

In [30]:
df_['customer_id']=  df_['customer_id'].apply(lambda x :  int( x[-16:],16))


In [31]:
df_[df_.customer_id ==4807831066164705350]

,t_dat,customer_id,article_id,price,sales_channel_id
196442,2018-09-24,4807831066164705350,0618480001,0.033881,2
196443,2018-09-24,4807831066164705350,0618480001,0.033881,2
196444,2018-09-24,4807831066164705350,0618480001,0.033881,2
196445,2018-09-24,4807831066164705350,0618480001,0.033881,2
196446,2018-09-24,4807831066164705350,0618480001,0.033881,2
...,...,...,...,...,...
196537,2018-09-24,4807831066164705350,0618480001,0.033881,2
196538,2018-09-24,4807831066164705350,0618480001,0.033881,2
196539,2018-09-24,4807831066164705350,0618480001,0.033881,2
196540,2018-09-24,4807831066164705350,0618480001,0.033881,2


In [32]:
df_['customer_id'].apply(lambda x: x.)

SyntaxError: invalid syntax (<ipython-input-32-5f58b2b450f7>, line 1)

In [33]:
df

,t_dat,customer_id,article_id,max_dat,diff_dat,ct
196442,2018-09-24,4807831066164705350,0618480001,2018-09-24,0,100
175867,2018-09-24,-4767130151449159669,0680374002,2018-09-24,0,55
60518,2018-09-21,-2936678038598836257,0456163028,2018-09-21,0,52
177445,2018-09-24,-8784462010106267426,0516000001,2018-09-24,0,36
49587,2018-09-21,-770382745269162367,0554772002,2018-09-21,0,33
...,...,...,...,...,...,...
48392,2018-09-20,-4545975550442162194,0671513003,2018-09-20,0,1
48393,2018-09-20,-4545975550442162194,0680186002,2018-09-20,0,1
48394,2018-09-20,-4545975550442162194,0679121002,2018-09-20,0,1
48395,2018-09-20,-4545975550442162194,0638642002,2018-09-20,0,1


In [ ]:
drop_cols = [col for col in dfi.columns if '_no' in col]
dfi =dfi.drop(drop_cols,axis=1 )

In [6]:
print('Unique Customer Count : {}'.format(dfu['customer_id'].nunique()))

In [7]:
# group 별 targeting

def create_generation_var():
    for age in range(1,10):
        # group 별 인원수
        globals()['{}0'.format(age)] = dfu[dfu.age.astype('str').str.startswith(str(age))]['customer_id'].values.tolist()
        globals()['{}0_gene_prod_group'.format(age)] =  dfi[dfi.article_id.isin(df[df.customer_id.isin(globals()['{}0'.format(age)])]['article_id'].values.tolist())]['product_group_name'].reset_index(drop=True)
        print('Age {}0 generation : {}'.format(age,len(dfu[dfu.age.astype('str').str.startswith(str(age))])))
    

In [8]:
## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] =='int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col]= df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col]= df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col]= df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col]= df[col].astype(np.int8)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
        end_mem = df.memory_usage().sum() / 1024**2
        if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
        return df    

In [9]:
def show_buy_group(df,exp_usr_id):
    
    buy_items = df[df.customer_id == exp_usr_id]['article_id'].values.tolist()
    # garment upper body 를 좋아하는 경향이 많다
    buy_items_group = dfi[dfi.article_id.isin(buy_items)]['product_group_name'].values.tolist()

    return print(buy_items_group)

In [10]:

def show_buy_ratio(df, exp_usr_id):
    
    merge_data = df[df.customer_id == exp_usr_id].merge(
        dfi[['article_id','prod_name','product_type_name','product_group_name','section_name']]
        , on = 'article_id')
    
    overall_buy_count = len(merge_data)
    
    # Womens 의류 개수의 합
    womens_ratio = merge_data.section_name.str.startswith('Womens').values.sum() / overall_buy_count
    
    # Mens 의류 개수의 합
    mens_ratio = merge_data.section_name.str.startswith('Mens').values.sum() / overall_buy_count

    # 자녀의 옷을 산 기록, 옷의 특징
    kids_ratio = merge_data.section_name.str.startswith(('Young Girl', 'Kids Girl')).values.sum() / overall_buy_count
    
     #전체 구매 개수
    # find duplicate buy item
    group_data = merge_data.groupby(['prod_name','product_type_name']).count().reset_index()
    
    duplicate_buy_items = group_data[group_data.article_id >1]['prod_name'].values 
    
    if len(duplicate_buy_items) >=1:
        duplicate_ratio =  len(duplicate_buy_items)/overall_buy_count
        
    return print(f'Womens Ratio : {womens_ratio} \\t Mens Ratio : {mens_ratio} \t Kids Ratio : {kids_ratio} \t    \t Duplicate items : {duplicate_buy_items}')

In [11]:
def show_generation_prefer_group(gene_prod_group_vc, age):
    
    trace = go.Bar( 
     x = gene_prod_group_vc.index,
     y = gene_prod_group_vc.values,
     marker = dict(
         color = random_color_generator(100),
         line = dict(color='rgb(8,48,107)', 
                     width = 1.5)
         ) 
    ,
    opacity = 0.7
     )

    data = [trace]    
    
    
    layout = go.Layout(
        title = f'{age} generation',
        margin = dict(
            l = 100
        ),
        xaxis = dict(
            title = 'Genre'
        ),
        yaxis = dict(
            title = 'Mean App size'
        ),  
        width = 800,
        height = 500
    )
    
    fig = go.Figure(data=data, layout=layout)
    po.iplot(fig)

In [12]:
def show_buy_items(df, exp_usr_id):
    
    data = df[df.customer_id ==exp_usr_id]
    
    # 구매한 아이템 이미지 
    f, ax = plt.subplots(5, 5, figsize=(30,15))
    i = 0
    k = 0
    for prod in data['article_id'].values.tolist():
        desc = dfi[dfi['article_id'] ==  prod]['detail_desc'].iloc[0]
        desc_list = desc.split(' ')

        for j, elem in enumerate(desc_list):
            if j > 0 and j % 5 == 0:
                desc_list[j] = desc_list[j] + '\n'
        desc = ' '.join(desc_list)
        section_nm = dfi[dfi['article_id'] ==  prod]['section_name'].iloc[0]
        img = mpimg.imread(f'../input/h-and-m-personalized-fashion-recommendations/images/{str(prod)[:3]}/0{int(prod)}.jpg')
        ax[i][k].imshow(img)
        ax[i][k].set_title(f'price: {data.price.values[0]:.2f}')
        ax[i][k].set_xticks([], [])
        ax[i][k].set_yticks([], [])
        ax[i][k].grid(False)
        ax[i][k].set_xlabel(section_nm, fontsize=10)
        k += 1 
        if k == 5:
            k  = 0
            i += 1
    plt.axis('off')
    plt.show()

In [13]:
def random_color_generator(number_of_colors):
    color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
                 for i in range(number_of_colors)]
    return color

In [14]:
# 20대 , 30대 , 40대로 묶기
fig , ax = plt.subplots(figsize=(15,7))
ax = sns.histplot(data=dfi ,y= 'index_name', color= 'orange')
ax.set_xlabel('count by index name')
ax.set_ylabel('index name')
plt.show()

# 20 대 여성이 많은 것으로 추정, 일반적으로 그렇다 쇼핑은



fig , ax = plt.subplots(figsize=(15,7))
ax = sns.histplot(data=dfi ,y= 'garment_group_name', hue='index_group_name',multiple='stack',  color= 'orange')
ax.set_xlabel('count by index name')
ax.set_ylabel('index name')
plt.show()

In [15]:
all_product_group_vc = dfi[dfi.article_id.isin(df['article_id'].values.tolist())]['product_group_name'].value_counts()
all_product_group_vc.index

In [16]:
all_product_group_vc.values

In [17]:
# overall product_group_name
# 상체에 관심이 많다 -> 모델 품질 측면에서는 diversity 한 추천 필요
plt.figure(figsize=(30,10))
plt.bar(x= all_product_group_vc.index, height=all_product_group_vc.values)
plt.xticks(all_product_group_vc.index,rotation=45 , size=20)
plt.show()

In [18]:

create_generation_var()

In [19]:
##

## Find popular item 

In [20]:
# generation popular item find
gene_prod_list = df[df.customer_id.isin(globals()['90'])]['article_id'].drop_duplicates().values.tolist()
dfi[dfi.article_id.isin(gene_prod_list)]['product_group_name']

## data split and grouping age

In [21]:
SEED=42
y = df['article_id'].values

x_train, x_valid, y_train, y_valid = train_test_split(df, y, test_size=0.2 ,random_state=SEED)
x_train_dfu = x_train.merge(dfu,on='customer_id')




## DATA EDA

In [22]:
x_train_dfu['age_str'] = x_train_dfu[x_train_dfu.age.str[:1]] 

In [23]:
x_train_dfu['age_group'] = x_train_dfu['age'].astype('str').str[:1]

In [24]:
merge_data = x_train_dfu.merge(dfi,on='article_id')

In [25]:
merge_data

In [26]:
x_train_desc = merge_data['detail_desc']

In [27]:
na_index = np.where(x_train_desc.isnull())[0]

In [28]:
na_index2 = np.where(x_train_dfu.age_group =='n')[0]

In [29]:
drop_index = set(na_index).union(set(na_index2))

In [30]:
x_train_desc

In [31]:
drop_index = list(drop_index)

In [32]:
drop_index[:5]

In [33]:
x_train_desc

In [34]:
x_train_desc= x_train_desc.drop(drop_index,axis=0)

In [35]:
indexing = list(set(np.arange(40000))- set(drop_index))

In [36]:
y = x_train_dfu.iloc[indexing,:]['age_group'].astype('int').values.tolist()

In [37]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(x_train_desc)
tfidf_vectorizer.idf_
x_train_desc_tfidf = tfidf_vectorizer.transform(x_train_desc).toarray()

In [38]:
x_train_desc_tfidf = x_train_desc_tfidf[indexing]

In [39]:
len(y)

In [40]:
# 그룹별 detail desc 의 tfidf 값 feature 가 logitic regression 으로 잘 분류되는 지 성능테스트

In [41]:
cols = sorted(tfidf_vectorizer.vocabulary_.items(), key=lambda x: x[1]) 

In [42]:
rows =  merge_data.drop(drop_index,axis=0)['article_id'].values.tolist()

In [43]:
cols  = [col[0] for col in cols]

In [44]:
merge_data.head()

In [45]:
merge_data

In [46]:
tfidf_df = pd.DataFrame(columns = cols, data = x_train_desc_tfidf)

In [47]:
tfidf_df

In [48]:
tfidf_df.loc[:,'shirt']

In [49]:
from sklearn.linear_model import LogisticRegression
# tfidf 값 만으로 logisticregression 하지 말고 feature 의 하나로 활용한다 tfidf 를
clf = LogisticRegression(random_state=0,solver='liblinear')
clf.fit(x_train_desc_tfidf,y)
pred =clf.predict(x_train_desc_tfidf)

In [50]:
# 분류 성능을 파악하여 tfidf 가 나이대를 잘 설명하는 지 확인한다

from sklearn.metrics import f1_score

In [51]:
y[:10]

In [52]:
pred[:10]

In [54]:
f1_score(y,pred, average='micro' )

In [61]:
# 가격대 , baseline 비슷한 유저대상  추천
df

In [63]:
!pip install cudf
import cudf

In [ ]:
print('RAPIDS version', cudf.__version__)

In [ ]:
#train = cudf.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')


In [59]:
dfu

In [57]:
df

In [ ]:
# average one of [None, 'micro', 'macro', 'weighted']

In [ ]:
# f1 score 값이 높아야 성능이 좋다
f1_score(y,pred,average ='micro')

In [ ]:
# 나이대별로 선호하는 낱말이 있다는 뜻 아닌가?

In [ ]:
 #오차행렬, 정확도, 정밀도, 재현율을 한번에 출력하기 위한 함수 정의

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    print('Confusion Matrix')
    print(confusion)
    print('정확도:{}, 정밀도:{}, 재현율:{}'.format(accuracy, precision, recall))

In [ ]:


# 사용자별 구매 주기
group_data = df.groupby('customer_id')


# 사용자별 구매주기
# exp_usr 
cnt = 0
for customer_id,data in group_data:
    exp_usr_itm = data.article_id.unique()
    exp_usr_id = data.customer_id.unique()[0]
    cnt += 1 
    if cnt == 1:
        break


# 연령대별 구매 데이터
# 그룹별 구매 빈도, 패턴

for age in range(1,9):
    
    age_group = x_train_dfu.groupby('age_group').get_group(str(age))
    print(f'Age {age}0 generation buy count : {len(age_group)}')
    
    # 나이대 별 구매 아이템 및 종류
    products =  age_group['article_id'].unique()

    print(f"Age {age}0 generation's unique product count : {len(products)}")
    # 30대는 20대에 비해 절반인데도 아이템 종류가 다양하다. 자녀의 아이템을 사거나 개인의 취향이 더 반영 된 듯한 모습
   

    # 나이대가 올라갈수록 다양한 아이템을 산다
    dfi[dfi.article_id.isin(products)]
    # 나이대별 feature의 다양성
    
    
    # 나이대별 tfidf 값으로 어떤 description 을 가진 옷을 선호하는 지 탐색하기

    x_train_desc = x_train_desc.dropna(axis=0)
     

    tfidf_vectorizer = TfidfVectorizer()

    tfidf_vectorizer.fit(x_train_desc)

    tfidf_vectorizer.idf_

    tfidf_vectorizer.transform(x_train_desc).toarray() 
    
    # catboost train 해보기
    
    

In [ ]:
for i , data in x_train_dfu.groupby('customer_id'):
    break

In [ ]:
# REDUCE MEMORY OF DATAFRAME
# hex문자열 int로 변환하기
df = df[['customer_id','article_id']]
#df.customer_id = df.customer_id.str[-16:].str.hex_to_int().astype('int64')
df['customer_id']=  df['customer_id'].apply(lambda x :  int( x[-16:],16))
df.article_id = df.article_id.astype('int32')
_ = gc.collect()

## 함께 구매한 아이템 찾기

In [ ]:
# FIND ITEMS PURCHASED TOGETHER
vc = df.article_id.value_counts()
pairs = {}
for j,i in enumerate(vc.index.values[1000:1032]):
    #if j%10==0: print(j,', ',end='')
    # i item을 구매한 유저
    USERS = df.loc[df.article_id==i.item(),'customer_id'].unique()
    # USERS 중에 i 아이템은 아닌 아이템의 value_counts() 를 위에서부터
    vc2 = df.loc[(df.customer_id.isin(USERS))&(df.article_id!=i.item()),'article_id'].value_counts()
    pairs[i.item()] = [vc2.index[0], vc2.index[1], vc2.index[2]]

In [ ]:
dfu['customer_id'] = dfu['customer_id'].apply(lambda x :  int( x[-16:],16))

In [ ]:
# sparse matrix 되므로 곤란하다
df.groupby(['customer_id','article_id']).count().reset_index()

In [ ]:
df['customer_id']=  df['customer_id'].apply(lambda x :  int( x[-16:],16))

In [ ]:
# 특정 아이템을 선호하는 그룹의 비율을 print해보면 grouping의 의미가 ? 
# user based filtering 을 하려면
# 특정 아이템을 구매한 유저들을 유사한 유저로
df[df.customer_id.isin(USERS)].merge(dfu, on='customer_id').age.plot.hist()

In [ ]:
for j , i in enumerate(vc.index.values[1000:1032]):
    print(j,i)
    break


In [ ]:
int(df.customer_id.str[-16:].values[0],16)

In [ ]:
df.customer_id.str[-16:].str.hex_to_int()

In [ ]:
# 연령대별 인기아이템 확인 겸 연령대별 feature를 추출하여 predict 

In [ ]:
# 해당 season에 유행한 아이템일 수 있으니 그것도 고려
# 저가 아이템이나 특정 가격의 아이템을 선호할 수도 있다


In [ ]:
vc.index.values[1000:1032]

In [ ]:
# FIND ITEMS PURCHASED TOGETHER
vc = df.article_id.value_counts()
pairs = {}

for j,i in enumerate(vc.index.values[1000:1032]):
    #if j%10==0: print(j,', ',end='')
    USERS = df.loc[df.article_id==i.item(),'customer_id'].unique()
    vc2 = df.loc[(df.customer_id.isin(USERS))&(df.article_id!=i.item()),'article_id'].value_counts()
    pairs[i.item()] = [vc2.index[0], vc2.index[1], vc2.index[2]]

In [ ]:
# https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/306007
# https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    # remove this case in advance
    # if not actual:
    #     return 0.0

    return score / min(len(actual), k)


def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [ ]:
x

In [ ]:
# 아이템에 선호하는 그룹대>

In [ ]:
# recommendation collaborative filtering in same group


In [ ]:
# local cv 설정 방법
#https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/308919


In [71]:
# group user basedfilteroing try
df = df.merge(dfu[['customer_id','age']])

In [70]:
twenty_generation = df[df.age.astype('str').str.startswith('2')]

In [69]:
# embedding model 로 test해보기
group_twenty = twenty_generation[['customer_id','article_id','t_dat']].groupby(['customer_id','article_id']).count().reset_index()

In [68]:
group_twenty = group_twenty.iloc[:500000]

In [ ]:
group_twenty.rename(columns={'t_dat':'count'},inplace=True)

In [ ]:
group_twenty 

In [66]:
pivot_twenty = group_twenty.pivot(index='customer_id',columns = 'article_id', values='count')

In [65]:
import gc 
del df
del twenty_generation
del group_twenty

gc.collect()

In [ ]:
pivot_twenty.dtypes

In [ ]:
# float dtype 은 메모리를 많이 잡아먹는다

In [ ]:
cols =  pivot_twenty.columns

In [ ]:
cols

In [ ]:
pivot_twenty[cols[0]].max() < np.iinfo(np.int32).max

In [ ]:
start_mem

In [ ]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] =='int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col]= df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col]= df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col]= df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col]= df[col].astype(np.int8)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
        end_mem = df.memory_usage().sum() / 1024**2
        if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
        return df    

In [ ]:
def reduce_memory_usage(df):
    
    for col in cols: 
        col_type = str(df[col].dtypes)
        if col_type.startswith('float'):
        

In [ ]:
# reduce memory 여기서 필요하다
reduce_memory_usage

In [ ]:
gc.collect()

In [ ]:
pivot_twenty.fillna(0,inplace=True)

In [64]:
pivot_twenty

In [ ]:
print(f'overall data lenth : {len(group_twenty)}')

In [ ]:
print(f"Unique article : {len(group_twenty['article_id'].unique())}")

In [ ]:
print(f"Unique customer : {len(group_twenty['customer_id'].unique())}")

In [ ]:
pivot_twenty

In [ ]:

df.t_dat = pd.to_datetime(df.t_dat)

In [ ]:
train = df.loc[df.t_dat <= pd.to_datetime('2020-09-15')]

In [ ]:
valid = df.loc[df.t_dat>= pd.to_datetime('2020-09-16')]

In [ ]:
# data trending 하기
# Step 1

In [ ]:
valid = valid.groupby('customer_id').article_id.apply(list).reset_index()
valid = valid.rename({'article_id':'prediction'},axis=1)
valid['prediction'] = \
        valid.prediction.apply(lambda x: ' '.join(['0'+str(k) for k in x]))


In [ ]:
 
df_sub = df_sub.set_index('customer_id').loc[valid.customer_id].reset_index() 


In [ ]:
df_sub

In [ ]:
mapk(valid.prediction.str.split(), df_sub.prediction.str.split(),k=12)

In [ ]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

In [ ]:
def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [ ]:
# 구매한 아이템들의 feature 를 해당 사용자와 mapping ? 

data

In [ ]:
x_train.merge(dfi,on='article_id')

In [ ]:
dfi['article_id'].nunique()

In [ ]:
show_buy_items( )

In [ ]:
# 같은 prod_name이면서 다른 product_code를 가진 아이템 show 하기

In [ ]:
# merge한 데이터에 tfidvectorizer를 해야 어떤 단어가 더 많이 등장하고 그런 것을 확인할 수 있을 듯 하기는 한데
# 메모리가 부족하다

In [ ]:
# 아이템 설명이 없는 row 제거

In [ ]:
# 각 행별로 tfidf 단어값이 나온다

In [ ]:
x_train

In [ ]:
tfidf_vectorizer = TfidfVectorizer()

In [ ]:
reduce_mem_usage(dfu)

In [ ]:
y_train

In [ ]:
merge_data = df.merge(dfi,on='article_id')

In [ ]:
# 연령대별 전체 아이템 그룹
# generation buy product_group_name
#gene_prod_group_vc = gene_prod_group.value_counts()

In [ ]:
reduce_memory_usage(dfi)

In [ ]:
reduce_memory_usage(dfu)

In [ ]:
dfi

In [ ]:
gene_prod_group_vc = globals()['10_gene_prod_group'].value_counts()

In [ ]:
# 전 연령대를 통틀어 상위 아이템이 관심이 많다
show_generation_prefer_group(gene_prod_group_vc , '90')

In [ ]:
exp_usr_id = dfu[dfu.age.astype('str').str.startswith('4')]['age'].values[5]

In [ ]:
data = df[df.customer_id ==exp_usr_id]

In [ ]:
data

In [ ]:
# text 정보에서 최대한 추측해보기
# desc에서 tfidf 하기

In [ ]:
# 아이템 속성 다양히 살펴보고 feature로 만들고 train valid 테스트해보기 ?


In [ ]:
# boost 
dfu[dfu.customer_id == exp_usr_id]['club_member_status']

In [ ]:
dfu[dfu.customer_id == exp_usr_id]['fashion_news_frequency']

In [ ]:
# 나이대 탐색
dfu[dfu.customer_id == exp_usr_id]['age'].values[0] in np.arange(40,50).tolist() 

In [ ]:
buy_items = df[df.customer_id == exp_usr_id]['article_id'].values.tolist()
# garment upper body 를 좋아하는 경향이 많다
dfi[dfi.article_id.isin(buy_items)]['product_group_name']

In [ ]:
# 40대 여성이 남편 옷 한번 , 자녀 옷, 주로 본인의 옷을 많이 샀다
# kids 의류를 산 True 인 개수의 합 
df[df.customer_id == exp_usr_id].merge(dfi[['article_id','prod_name','product_type_name','product_group_name','section_name']] , on = 'article_id').section_name.str.startswith('Kids').values.sum()

In [ ]:
show_buy_ratio(df,exp_usr_id)

In [ ]:
# 그리고 선호하는 옷의 이미지 특성을 탐색해서 유사한 사용자의 옷을 추천하거나 옷의 특성을 탐색해야지, 여백이 많은 옷을 좋아하는 지 화려한 옷을 좋아하는 지 등의

In [ ]:
# 좋아하는 옷을 2벌 3벌 사는 유형이 있다. 
# 그리고 그냥 나처럼 가끔 가서 내가 좋아보이는 옷을 사는 유형
dfi[dfi.article_id.isin(buy_items2)]

In [ ]:

# 반복 구매하는 아이템에 대해 가중치를 주고 반복 구매 체크하고 어떤 아이템을 반복 구매했는 지 체크
show_buy_group(df,exp_usr_id)
# 40대 유저는 tailoring 을 많이산다


In [ ]:
show_buy_items(df,exp_usr_id) 

In [ ]:
# 40대 사용자 중 랜덤하게 한명 선택
exp_usr_id = np.random.choice( dfu[dfu['age'].astype('str').str.startswith('4')]['customer_id'].values.tolist() , 1 )

In [ ]:
def show_buy_group(df,exp_usr_id):
    
    buy_items = df[df.customer_id == exp_usr_id]['article_id'].values.tolist()
    # garment upper body 를 좋아하는 경향이 많다
    buy_items_group = dfi[dfi.article_id.isin(buy_items)]['product_group_name'].values.tolist()
    k
    return print(buy_items_group)

In [ ]:
show_buy_ratio(df,exp_usr_id )

In [ ]:
show_buy_ratio(df,exp_usr_id)
# 반복 구매하는 아이템에 대해 가중치를 주고 반복 구매 체크하고 어떤 아이템을 반복 구매했는 지 체크

In [ ]:
show_buy_group(df,exp_usr_id)
# 40대 유저는 tailoring 을 많이산다

In [ ]:
show_buy_items(df,exp_usr_id) 

In [ ]:
# 중복 비율
df[df.customer_id.isin(globals()['{}0'.format(age)])]['article_id']

In [ ]:
df[df.customer_id.isin(globals()['{}0'.format(age)])]['article_id'].drop_duplicates()

In [ ]:
a

In [ ]:
gene_prod_group

In [ ]:
import random

In [ ]:
dfu.age.plot.hist()

In [ ]:
for idx , prod in enumerate(data['article_id'].values.tolist()):
    print(idx, prod)

In [ ]:
for idx, prod in enumerate(data['article_id'].values.tolist()):
    print(idx)

In [ ]:
 desc = dfi[dfi['article_id'] ==  prod]['detail_desc'].iloc[0]

In [ ]:
# 구매한 아이템 이미지 
f, ax = plt.subplots(5, 5, figsize=(30,15))
i = 0
k = 0
for prod in data['article_id'].values.tolist():
    desc = dfi[dfi['article_id'] ==  prod]['detail_desc'].iloc[0]
    desc_list = desc.split(' ')

    for j, elem in enumerate(desc_list):
        if j > 0 and j % 5 == 0:
            desc_list[j] = desc_list[j] + '\n'
    desc = ' '.join(desc_list)
    section_nm = dfi[dfi['article_id'] ==  prod]['section_name'].iloc[0]
    img = mpimg.imread(f'../input/h-and-m-personalized-fashion-recommendations/images/{str(prod)[:3]}/0{int(prod)}.jpg')
    ax[i][k].imshow(img)
    ax[i][k].set_title(f'price: {data.price.values[0]:.2f}')
    ax[i][k].set_xticks([], [])
    ax[i][k].set_yticks([], [])
    ax[i][k].grid(False)
    ax[i][k].set_xlabel(section_nm, fontsize=10)
    k += 1 
    if k == 5:
        k  = 0
        i += 1
plt.axis('off')
plt.show()

In [ ]:
cols = df.columns.tolist()

In [ ]:
# in 연산자는 list에서만 
# df[cols[3]].dtypes in 'float64' x 

In [ ]:
reduce_mem_usage(df)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df

In [ ]:
x_train , x_valid , y_train ,y_valid = train_test_split(X, Y, test_size=0.2, stratify=Y)

In [ ]:
col_type in numerics

In [ ]:
# 연령대 분포
plt.figure(figsize=(20,7))
sns.histplot(dfu['age'])

In [ ]:
dfi['article_id'].nunique()

In [ ]:
# 그룹별 구매 아이템의 특징


In [ ]:
df

![](http://)

In [ ]:
df_sub

In [ ]:
# article_id count 인기 품목
df.groupby('article_id').count().reset_index()

In [ ]:
# 가장 많이 팔린 품목 또한 여성 제품
dfi[dfi['article_id'] == '0108775015']